# 🧪 Lab Supply Calculator

Welcome! Use this tool to:

- Select a protocol
- Enter your sample size
- Click "Run All"
- Receive a downloadable CSV with supply needs and reorder status

🔒 **Note**: You’ll need a GitHub access token to access the files. Please see Aubrie or Vicki for this.

---

## 🧭 Instructions

1. Enter your **GitHub token**
2. Enter the **sample size**
3. Choose the **protocol**
4. Click **Runtime → Run All**
5. Download the CSV report when prompted

---

### Example:
- Token: `ghp_XXXXXXXXXXXXXXXXXXXX`
- Sample size: `50`



In [61]:

#@title Experiment Setup
#@markdown Select a protocol and sample size:
# @title
from IPython.display import display
import ipywidgets as widgets

# Token input
token_input = widgets.Text(
    value='ghp_YourTokenHere',  # Replace this value with blank if preferred
    description='GitHub token:',
    layout=widgets.Layout(width='80%')
)

# Sample size input
sample_input = widgets.IntText(
    value=50,
    description='Sample size:',
    disabled=False
)

# Protocol selection dropdown (static example list)
protocol_dropdown = widgets.Dropdown(
    options=[
        "test_protocol_check/protocols/dna_extraction_mn.yaml",
        "test_protocol_check/protocols/pcr_setup.yaml"
    ],
    description='Protocol:',
    layout=widgets.Layout(width='95%')
)

# Display all widgets
display(token_input, sample_input, protocol_dropdown)

# Function to wait for token widget change
async def wait_for_input(widget):
    print("⏸️ Waiting for token input...")
    future = asyncio.get_event_loop().create_future()

    def on_change(change):
        if change['new']:
            future.set_result(change['new'])

    widget.observe(on_change, names='value')
    await future
    print("✅ Token received.")

# Pause here until the user enters a value
await wait_for_input(token_input)

# Now continue with the rest of your code
token = token_input.value
print("Token entered:", token[:4] + "...")



Text(value='ghp_YourTokenHere', description='GitHub token:', layout=Layout(width='80%'))

IntText(value=50, description='Sample size:')

Dropdown(description='Protocol:', layout=Layout(width='95%'), options=('test_protocol_check/protocols/dna_extr…

⏸️ Waiting for token input...


NameError: name 'asyncio' is not defined

In [ ]:
#@title Installing packages
#@markdown Installing packages


# Step 1. Install required packages (if not already installed)
!pip install pyyaml

# Import libraries
import requests
import csv
import yaml
import pandas as pd
import os
from io import StringIO
from google.colab import files
from IPython.display import display
import ipywidgets as widgets


In [ ]:
# @title Defining GitHub Repo configuration

# 2. Define fixed GitHub repo configuration
GITHUB_USER = "aono87"
REPO_NAME = "inventory_test"
BRANCH = "main"
INVENTORY_PATH = "test_protocol_check/inventory/inventory.csv"
PROTOCOL_DIR = "test_protocol_check/protocols"


In [ ]:
#@title Fetching protocols from GitHub
# 4. Function to fetch protocol file list from GitHub
def get_protocol_list(token, user, repo, path, branch="main"):
    url = f"https://api.github.com/repos/{user}/{repo}/contents/{path}?ref={branch}"
    headers = {"Authorization": f"token {token}"}
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    files = r.json()
    return [f['path'] for f in files if f['name'].endswith('.yaml')]

# Fetch protocol paths
protocols = get_protocol_list(
    token_input.value,
    GITHUB_USER,
    REPO_NAME,
    PROTOCOL_DIR,
    BRANCH
)

In [ ]:
#5. Define helper functions
def github_raw_url(path):
    return f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/{BRANCH}/{path}"

def fetch_file(path, token):
    url = github_raw_url(path)
    headers = {"Authorization": f"token {token}"}
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    return r.text

def load_inventory(text):
    inv = {}
    reader = csv.DictReader(StringIO(text.lstrip('\ufeff')))
    for row in reader:
        item = row['Item'].strip()
        inv[item] = {
            'unit': row['Unit'].strip(),
            'stock': float(row['Stock Quantity']),
            'threshold': float(row['Reorder Threshold'])
        }
    return inv

def load_protocol(text):
    return yaml.safe_load(text)

def calculate_needs(protocol_data, sample_count):
    return {item: qty * sample_count for item, qty in protocol_data['supplies_per_sample'].items()}


In [ ]:
# 6. Fetch and parse files
inventory_text = fetch_file(INVENTORY_PATH, token_input.value)
protocol_text = fetch_file(protocol_dropdown.value, token_input.value)

inventory = load_inventory(inventory_text)
protocol = load_protocol(protocol_text)
needs = calculate_needs(protocol, sample_input.value)

# Create report including per-sample use
report_rows = []

# Pull per-sample values from protocol YAML
per_sample = protocol["supplies_per_sample"]

for item, total_required in needs.items():
    per_unit = per_sample.get(item, "N/A")
    inv = inventory.get(item)

    if not inv:
        report_rows.append({
            "Item": item,
            "Per Sample": per_unit,
            "Need": total_required,
            "Stock": None,
            "Status": "MISSING",
            "Reorder": "⚠️"
        })
    else:
        stock = inv['stock']
        threshold = inv['threshold']
        status = "OK" if stock >= total_required else "LOW"
        reorder_flag = "YES" if (stock - total_required) < threshold else "NO"
        report_rows.append({
            "Item": item,
            "Per Sample": per_unit,
            "Need": round(total_required, 2),
            "Stock": round(stock, 2),
            "Status": status,
            "Reorder": reorder_flag
        })

# Convert to DataFrame
df = pd.DataFrame(report_rows)

# Show the updated report in notebook
print(f"\n📋 Supply Report for {sample_input.value} samples using {protocol_dropdown.value}")
print(df.to_string(index=False))


In [ ]:
# 7. Save and download CSV report with metadata

# Extract just the filename from the protocol path
protocol_name = os.path.basename(protocol_dropdown.value)

# Prepare metadata rows
metadata_rows = pd.DataFrame([
    {"Item": "Protocol:", "Need": protocol_name, "Stock": "", "Status": "", "Reorder": ""},
    {"Item": "Sample Size:", "Need": sample_input.value, "Stock": "", "Status": "", "Reorder": ""},
    {},  # Blank row for separation
])

# Path to save CSV
output_path = "/content/supply_report.csv"

# Write metadata (no header)
metadata_rows.to_csv(output_path, mode='w', index=False, header=False)

# Append main dataframe with headers
df.to_csv(output_path, mode='a', index=False, header=True)

# Trigger download
print("\n✅ CSV with metadata saved. Downloading...")
files.download(output_path)